In [2]:
import random
import pandas as pd
import numpy as np
from scipy.stats import loguniform
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split


from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import VotingClassifier

from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')

# 임의데이터 생성

최초 1회 실행

In [22]:
# 전체 숙소 특징
n = 5000
acc = pd.DataFrame({
    '숙소 ID' : range(1,n+1),
    '1박 당 가격' : np.random.randint(50000, 1000000, n),
    '침실 수' : np.random.randint(1, 5, n),
    '침대 수' : np.random.randint(1, 10, n),
    '인터넷 유무' : np.random.randint(0, 2, n),
    'TV 유무' : np.random.randint(0, 2, n),
    '냉장고 유무' : np.random.randint(0, 2, n),
    '에어컨 유뮤' : np.random.randint(0, 2, n),
    '수영장 유무' : np.random.randint(0, 2, n),
    '숙소 유형' : random.choices(['단독 주택', '아파트', '다세대 주택','호텔','모텔'], k=n)
})

# 사용자
o = 3000
user = pd.DataFrame({
    'ID' : range(1,o+1),
    '나이' : random.choices([20,30,40,50,60,70,80], k=o),
    '성별' : random.choices(['남성', '여성'], k=o) ,
    '직업' : random.choices(['학생', '직장인','없음'], k=o)
})
# 다른(모든) 사용자의 예약 내역
reservation_data = []
search_data = []

for uid in user['ID']:
    n_res = np.random.randint(1, 6)  # 최근 예약 5개
    n_srch = np.random.randint(1, 6)  # 최근 검색 5개

    # 숙소 무작위 선택 (중복 허용)
    reserved = acc.sample(n=n_res, replace=True).copy()
    reserved['user_id'] = uid
    reserved['type'] = 'reservation'

    searched = acc.sample(n=n_srch, replace=True).copy()
    searched['user_id'] = uid
    searched['type'] = 'search'

    # 누적
    reservation_data.append(reserved)
    search_data.append(searched)
search_data = pd.concat(search_data)
reservation_data = pd.concat(reservation_data)
df = pd.DataFrame(pd.concat([reservation_data,search_data]))
df.sort_values(by='user_id', inplace=True)
df.reset_index(drop=True, inplace=True)

In [25]:
acc.to_csv('acc.csv', index=False)
user.to_csv('user.csv', index=False)
df.to_csv('df.csv', index=False)

# 모델 파라미터 생성

In [9]:
# RandomForestClassifier모델 생성
rf_param = {
    'max_depth' : range(1,11),
    'n_estimators' : range(1,101),
    'max_features' : ['auto', 'sqrt', 'log2']
}
rf = RandomForestClassifier()
rf_rs = RandomizedSearchCV(estimator=rf, param_distributions=rf_param, scoring='accuracy',cv=10,n_jobs=-1,n_iter=10)
# LogisticRegression모델 생성
lr_param = {
    'C' : loguniform(1e-4, 1e4),
    'tol' : loguniform(1e-6, 1e-2),
    'max_iter' : [100, 1000, 10000]
}
lr = LogisticRegression()
lr_rs = RandomizedSearchCV(estimator=lr,param_distributions=lr_param,scoring='accuracy',cv=10,n_jobs=-1,n_iter=30)
# SVM 모델 생성
svm_param = {
    'C' : loguniform(1e-4, 1e4),
    'kernel' : ['rbf', 'sigmoid'],
    'gamma' : ['scale', 'auto']
}
svm = SVC( probability=True)
svm_rs = RandomizedSearchCV(estimator=svm,param_distributions=svm_param,scoring='accuracy',cv=10,n_jobs=-1,n_iter=15)

# ID 입력 후 추천 시스템 작동

In [26]:
acc = pd.read_csv('/content/acc.csv')
user = pd.read_csv('/content/user.csv')
df = pd.read_csv('/content/df.csv')

In [23]:
class RecommenderSystem:
    def __init__(self, user, df, acc):
        self.user = user.copy()
        self.df = df.copy()
        self.acc = acc.copy()

        # 모델 초기화
        self.rf = RandomForestClassifier(random_state=1)
        self.lr = LogisticRegression(max_iter=1000, random_state=1)
        self.svm = SVC(kernel='rbf', C=10, gamma=1, probability=True, random_state=1)
        self.esb = None
        self.accuracy = None

    def recommend(self, user_id):
        ### 유사 사용자 추출
        target_user = self.user[self.user['ID'] == user_id].iloc[0]

        def count_similar_features(row):
            same_age = row['나이'] == target_user['나이']
            same_gender = row['성별'] == target_user['성별']
            same_job = row['직업'] == target_user['직업']
            return sum([same_age, same_gender, same_job])

        self.user['겹치는 특성 수'] = self.user.apply(count_similar_features, axis=1)
        similar_users = self.user[self.user['겹치는 특성 수'] >= 2]
        similar_user_ids = similar_users['ID'].tolist()

        # 사용자 상호작용 정보
        reserved = self.df[(self.df['user_id'].isin(similar_user_ids)) & (self.df['type'] == 'reservation')]['숙소 ID'].unique()
        searched = self.df[(self.df['user_id'].isin(similar_user_ids)) & (self.df['type'] == 'search')]['숙소 ID'].unique()

        acc_df = self.acc.copy()
        acc_df['class'] = 0
        acc_df.loc[acc_df['숙소 ID'].isin(searched), 'class'] = 1
        acc_df.loc[acc_df['숙소 ID'].isin(reserved), 'class'] = 2

        # 전처리
        y = acc_df['class']
        x = acc_df.drop(columns=['class', '숙소 ID'])
        x_ohe = pd.get_dummies(x, columns=['숙소 유형'], drop_first=True).astype(int)
        sc = StandardScaler()
        num_col = x.select_dtypes(include=['int64', 'float64']).columns
        x_ohe[num_col] = sc.fit_transform(x_ohe[num_col])

        x_train, x_test, y_train, y_test = train_test_split(x_ohe, y, test_size=0.2, random_state=42)

        # 모델 학습
        self.rf.fit(x_train, y_train)
        self.lr.fit(x_train, y_train)
        self.svm.fit(x_train, y_train)

        self.esb = VotingClassifier(estimators=[('rf', self.rf), ('lr', self.lr), ('svm', self.svm)], voting='soft')
        self.esb.fit(x_train, y_train)

        # 예측 및 추천 점수
        y_proba = self.esb.predict_proba(x_test)
        y_pred = np.argmax(y_proba, axis=1)
        self.accuracy = accuracy_score(y_test, y_pred)

        res = self.esb.predict_proba(x_ohe)[:,2]
        sch = self.esb.predict_proba(x_ohe)[:,1]
        acc_df['proba'] = res + 0.5 * sch

        acc_df.sort_values(by='proba', ascending=False, inplace=True)
        acc_df.reset_index(drop=True, inplace=True)

        return acc_df.iloc[:10, :-2]

    def evaluate(self):
        return self.accuracy

In [24]:
model = RecommenderSystem(user, df, acc)
inp = int(input("ID를 입력: "))
top10 = model.recommend(user_id=inp)
print('정확도 :',model.evaluate())
top10

ID를 입력: 123
정확도 : 0.418


,숙소 ID,1박 당 가격,침실 수,침대 수,인터넷 유무,TV 유무,냉장고 유무,에어컨 유뮤,수영장 유무,숙소 유형
0,4409,437393,2,2,1,0,1,0,0,다세대 주택
1,3263,437051,2,2,1,0,1,0,0,다세대 주택
2,2288,436073,4,9,0,0,1,0,1,다세대 주택
3,2107,154217,4,9,0,0,1,0,1,다세대 주택
4,2129,901586,4,2,0,0,0,1,0,다세대 주택
5,1814,423916,4,6,1,0,0,0,1,아파트
6,852,467391,1,8,1,0,1,1,1,다세대 주택
7,858,648467,4,4,0,0,1,0,0,모텔
8,4138,362944,2,9,0,1,1,1,0,단독 주택
9,322,120710,3,7,1,0,0,1,0,단독 주택
